<a href="https://colab.research.google.com/github/strongeryoung/class_SmartDataAnalysis/blob/main/Tripadvisor_crawler_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

import requests

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import subprocess
import shutil
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup
import time
import datetime

import os
import re
import chromedriver_autoinstaller
from selenium.webdriver import ActionChains
import numpy as np

In [ ]:
def getSessionInfo(driver,url):

    # 해당 사이트 이동
    driver.get(url)
    time.sleep(5)

    #  request Session을 만든다
    s = requests.Session()
    # 쿠키 업데이트로 세션을 로그인된걸로 알게 한다
    for cookie in driver.get_cookies():
        c = {cookie['name'] : cookie['value']}
        s.cookies.update(c)
    return (s)

In [ ]:
data = pd.read_excel(r'C:\Users\MetaM\PycharmProjects\jobposting\hotel_url.xlsx')
data = data.drop(columns= 'Unnamed: 0')
url_list = data['url'].values.tolist()

In [ ]:
chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  # 크롬 버전을 확인한다.

driver_path = r"C:\Users\MetaM\PycharmProjects\jobposting\120\chromedriver.exe"

# driver_path = f'./{chrome_ver}/chromedriver.exe'
if os.path.exists(driver_path):
    print(f"chromedriver is installed: {driver_path}")  # 있는 버전을 쓴다.
else:
    print(f"install the chrome driver(ver: {chrome_ver})")  # 크롬을 최신 버전으로 설치한다.
    chromedriver_autoinstaller.install(True)

options = webdriver.ChromeOptions()  # 크롬 옵션을 추가한다.
# options.add_argument('headless')
options.add_experimental_option("detach", True)  # 크롬 안 꺼지는 옵션 추가
options.add_experimental_option("excludeSwitches", ["enable-logging"])  # 크롬 안 꺼지는 옵션 추가
browser = webdriver.Chrome(driver_path, options=options)  # 크롬 드라이버를 할당

chromedriver is installed: C:\Users\MetaM\PycharmProjects\jobposting\120\chromedriver.exe


C:\Users\MetaM\AppData\Local\Temp\ipykernel_24672\4034276062.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(driver_path, options=options)  # 크롬 드라이버를 할당


In [ ]:
url = url_list[0]
browser.get(url)
browser.maximize_window()
browser.implicitly_wait(3)

In [ ]:
page_source = browser.page_source
soup = BeautifulSoup(page_source, 'html.parser')
review = soup.find_all('div', attrs={'class': 'fIrGe _T'})

In [ ]:
review_list = []
for i in range(1,7):
    review_txt = review[i].get_text()
    review_list.append(review_txt)

In [ ]:
pd.DataFrame(review_list)

,0
0,객실이 낡고 카페트도 더럽고 복도에는 담배냄새가 심하게 나는 경우가 있었음. 디즈니...
1,Thank you for taking your time and sharing you...
2,차라리 도쿄 시내에 방잡고 지하철 타고 다니세요...어차피 모노레일 타야해서...여...
3,디즈니 이용을 염두에 둔다면 나쁘지 않습니다. 마이하마역까지 셔틀 운행을 하므로 도...
4,"싱글침대를 붙여, 넓은 침대공간을 제공받았는데 아이들이 너무 좋아하였습니다. 이외에..."
5,디즈니랜드 접근성이 좋고 아이들에 대한 세심한 배려가 곳곳에서 보였습니다. 다시 가...


In [ ]:
reviews_num = ['', 5, 10, 15, 20, 25, 30]

review_list = []
dt_list = []

for reviews in reviews_num:

    URL = f'https://www.tripadvisor.co.kr/Hotel_Review-g14134868-d304292-Reviews-or{reviews}-Hilton_Tokyo_Bay-Maihama_Urayasu_Chiba_Prefecture_Kanto.html#REVIEWS'

    ## 세션 인포메니션을 업데이트 시킨다
    s = getSessionInfo(browser, URL)

    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36',
        'referer': 'https://www.tripadvisor.co.kr/'
    }

    s.headers.update(headers)

    browser.get(URL)
    browser.maximize_window()
    browser.implicitly_wait(3)

    page_source = browser.page_source

    soup = BeautifulSoup(page_source, 'html.parser')
    review = soup.find_all('div', attrs={'class': 'fIrGe _T'})
    dates = soup.find_all('div', attrs={'class': 'cRVSd'})

    for i in range(1,len(review)):
        review_txt = review[i].get_text()
        review_list.append(review_txt)

    for j in range(len(dates)):
        date_txt = dates[j].get_text()
        dt_list.append(date_txt)

In [ ]:
review_list[5]

'디즈니랜드 접근성이 좋고 아이들에 대한 세심한 배려가 곳곳에서 보였습니다. 다시 가게 된다면 해피매직룸에 또 한 번 투숙하고 싶어요 :)'